In [0]:
from google.colab import drive
drive.mount('/content/drive')

root_dir = '/content/drive/My Drive/'

%cd "{root_dir}"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


In [0]:
!git clone https://github.com/xelav/Insider-Threat-CERT /root/repo

import sys
sys.path.append("/root/repo")

Cloning into '/root/repo'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 120 (delta 75), reused 97 (delta 56), pack-reused 0
Receiving objects: 100% (120/120), 90.95 KiB | 500.00 KiB/s, done.
Resolving deltas: 100% (75/75), done.


In [0]:
!pip install pytorch-ignite

     |████████████████████████████████| 112kB 438kB/s 


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
import shutil
from tqdm.notebook import tqdm

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.metrics import Accuracy, Loss

from src.models import InsiderClassifier, LSTM_Encoder
from src.params import get_params
from src.dataset import CertDataset, create_data_loaders
from src.lstm_trainer import *

%load_ext autoreload
%autoreload 2

# TODO:

* Больше детерминизма



In [0]:
# output_dir = Path(r'C:\Users\Mvideo\Google Drive\Datasets\CERT_output')
# answers_dir = Path(r"C:/Users/Mvideo/Downloads/answers")

output_dir = Path(f'{root_dir}/Datasets/CERT_output')
answers_dir = Path(f"{root_dir}/Datasets/CERT/answers")
main_answers_file = answers_dir / "insiders.csv"

assert(output_dir.is_dir())
assert(answers_dir.is_dir())

run_name = 'lstm/embedding_test'

log_dir = output_dir / 'logs' / run_name
checkpoint_dir = output_dir / 'checkpoints' / run_name

# assert(not log_dir.is_dir())
# assert(not checkpoint_dir.is_dir())

if log_dir.is_dir():
    shutil.rmtree(log_dir)
if checkpoint_dir.is_dir():
    shutil.rmtree(checkpoint_dir)

In [0]:
actions, targets = CertDataset.prepare_dataset(output_dir / 'aggregated.pkl', main_answers_file, min_length=50, max_length=200)

# Train

TODO:
* Эксперименты с LR Scheduling

In [0]:
cert_dataset = CertDataset(actions, targets)
train_loader, val_loader = create_data_loaders(cert_dataset, validation_split=0.3, random_seed=0, batch_size=1024)

params = get_params()

device = 'cuda'

In [0]:
lstm_encoder = LSTM_Encoder(params['model']['lstm_encoder'])
criterion = nn.NLLLoss()
optimizer = optim.Adam(lstm_encoder.parameters())

train_engine = create_supervised_trainer_lstm(
                                        lstm_encoder, optimizer, criterion, device=device,
                                        prepare_batch=prepare_batch_lstm,
                                        log_dir=log_dir.as_posix(),
                                        checkpoint_dir=checkpoint_dir,
                                        checkpoint_every=500,
                                        tensorboard_every=10,
                                       )

val_engine = create_supervised_evaluator_lstm(
        lstm_encoder, device=device,
        prepare_batch=prepare_batch_lstm,
        metrics={},
        criterion=criterion,
        log_dir=log_dir.as_posix(),
)

@train_engine.on(Events.STARTED)
def log_training_results(trainer):
    print('Initial validation run:')
    val_engine.train_epoch = 0
    val_engine.run(val_loader)

@train_engine.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    print('Validation run:')
    val_engine.train_epoch = train_engine.state.epoch
    val_engine.run(val_loader)


In [0]:
train_engine.run(train_loader, max_epochs=10)

Initial validation run:



Validation Results - Avg loss: 4.178010, Accuracy: 0.000887, Non-Pad-Accuracy: 0.007730



Epoch results - Avg loss: 1.866784, Accuracy: 0.793244, Non-Pad-Accuracy: 0.024437
Validation run:



Validation Results - Avg loss: 0.542273, Accuracy: 0.885986, Non-Pad-Accuracy: 0.649159



Epoch results - Avg loss: 0.419580, Accuracy: 0.899682, Non-Pad-Accuracy: 0.606825
Validation run:


...


Epoch results - Avg loss: 0.157149, Accuracy: 0.967291, Non-Pad-Accuracy: 0.720845
Validation run:



Validation Results - Avg loss: 0.154710, Accuracy: 0.967496, Non-Pad-Accuracy: 0.721163



Epoch results - Avg loss: 0.153329, Accuracy: 0.967691, Non-Pad-Accuracy: 0.723323
Validation run:



Validation Results - Avg loss: 0.150740, Accuracy: 0.967779, Non-Pad-Accuracy: 0.722838


State:
	iteration: 760
	epoch: 10
	epoch_length: 76
	max_epochs: 10
	output: <class 'dict'>
	batch: <class 'dict'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: 12

In [0]:
%ls "{checkpoint_dir}"

checkpoint_500.pth  final_model_760.pth


In [0]:
from google.colab import files
files.download(checkpoint_dir / 'final_model_760.pth') 

In [0]:
%load_ext tensorboard
%tensorboard --logdir "{log_dir}"

# Prediction exploration

TODO:
- decode class numbers

In [0]:
batch = next(iter(val_loader))

In [0]:
x, y = prepare_batch_lstm(batch)

In [0]:
lstm_encoder.train()

LSTM_Encoder(
  (lstm_encoder): LSTM(64, 40, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (decoder): Linear(in_features=40, out_features=64, bias=True)
  (log_softmax): LogSoftmax()
)

In [0]:
y.argmax(dim=2)[2]

tensor([54, 54, 54, 54, 54, 54, 54, 54, 62, 54, 54, 44,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0])

In [0]:
lstm_encoder(x.to('cuda')).exp().argmax(dim=2)[2]

tensor([54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0], device='cuda:0')

In [0]:
lstm_encoder(x.to('cuda')).exp().argmax(dim=2).unique()

tensor([ 0, 38, 43, 44, 48, 54, 59, 60], device='cuda:0')